In [11]:
import together, os
from together import Together
import json

# Paste in your Together AI API Key or load it
# TOGETHER_API_KEY = os.environ.get("TOGETHER_API_KEY")
TOGETHER_API_KEY = "key"
# print(TOGETHER_API_KEY)


with open('RAG_data/database/merged_data.json', 'r') as file:
    rag_data = json.load(file)

rag_data[:1]



[{'training_goal': 'Focused on developing offensive strategies including mastering accurate shooting, effective dribbling to evade opponents, and practicing different pass types (long and high passes) to break defensive lines.',
  'num_agents': 4,
  'reward_function': 'import gym\nimport numpy as np\nclass CheckpointRewardWrapper(gym.RewardWrapper):\n    """A wrapper that adds a task-specific reward for offensive strategies focused on shooting, dribbling, and passing."""\n\n    def __init__(self, env):\n        super().__init__(env)\n        self.sticky_actions_counter = np.zeros(10, dtype=int)  # Tracks sticky actions for each agent\n\n    def reset(self):\n        """Reset the reward wrapper state along with the environment."""\n        self.sticky_actions_counter = np.zeros(10, dtype=int)\n        return self.env.reset()\n\n    def get_state(self, to_pickle):\n        """Get the state of the wrapper with additional wrapper states."""\n        to_pickle[\'sticky_actions_counter\'] = 

In [6]:
len(rag_data)

30

{
    "training_goal": "Focused on developing offensive strategies including mastering accurate shooting, effective dribbling to evade opponents, and practicing different pass types (long and high passes) to break defensive lines.",
    "num_agents": 4,
    "reward_function": "import gym\nimport numpy as np\nclass CheckpointRewardWrapper(gym.RewardWrapper):\n    \"\"\"A wrapper that adds a task-specific reward for offensive strategies focused on shooting, dribbling, and passing.\"\"\"\n\n    def __init__(self, env):\n        super().__init__(env)\n        self.sticky_actions_counter = np.zeros(10, dtype=int)  # Tracks sticky actions for each agent\n\n    def reset(self):\n        \"\"\"Reset the reward wrapper state along with the environment.\"\"\"\n        self.sticky_actions_counter = np.zeros(10, dtype=int)\n        return self.env.reset()\n\n    def get_state(self, to_pickle):\n        \"\"\"Get the state of the wrapper with additional wrapper states.\"\"\"\n        to_pickle['sticky_actions_counter'] = self.sticky_actions_counter\n        return self.env.get_state(to_pickle)\n\n    def set_state(self, state):\n        \"\"\"Set the state of the wrapper from the pickle object.\"\"\"\n        from_pickle = self.env.set_state(state)\n        self.sticky_actions_counter = from_pickle['sticky_actions_counter']\n        return from_pickle\n\n    def reward(self, reward):\n        \"\"\"Custom reward function to encourage effective offensive plays.\"\"\"\n        observation = self.env.unwrapped.observation()\n        components = {\"base_score_reward\": reward.copy(),\n                      \"shooting_reward\": [0.0] * len(reward),\n                      \"dribbling_reward\": [0.0] * len(reward),\n                      \"passing_reward\": [0.0] * len(reward)}\n\n        if observation is None:\n            return reward, components\n\n        assert len(reward) == len(observation)\n        \n        for rew_index in range(len(reward)):\n            o = observation[rew_index]\n            is_shooting = o['sticky_actions'][9]  # Assuming index 9 is related to shooting action\n            effective_dribble = o['sticky_actions'][8]  # Assuming index 8 is dribbling\n            is_passing = o['sticky_actions'][7]  # Assuming index 7 is passing\n\n            if is_shooting:\n                components[\"shooting_reward\"][rew_index] = 0.1\n                reward[rew_index] += components[\"shooting_reward\"][rew_index]\n            \n            if effective_dribble:\n                components[\"dribbling_reward\"][rew_index] = 0.05\n                reward[rew_index] += components[\"dribbling_reward\"][rew_index]\n            \n            if is_passing:\n                components[\"passing_reward\"][rew_index] = 0.03\n                reward[rew_index] += components[\"passing_reward\"][rew_index]\n        \n        return reward, components\n\n    def step(self, action):\n        \"\"\"Step the environment and apply the reward wrapper.\"\"\"\n        observation, reward, done, info = self.env.step(action)\n        reward, components = self.reward(reward)\n        info[\"final_reward\"] = sum(reward)\n        for key, value in components.items():\n            info[f\"component_{key}\"] = sum(value)\n        obs = self.env.unwrapped.observation()\n        self.sticky_actions_counter.fill(0)\n        for agent_obs in obs:\n            for i, action in enumerate(agent_obs['sticky_actions']):\n                self.sticky_actions_counter[i] += action\n        return observation, reward, done, info\n",
    "evaluation": "Yes",
    "suggestions": "1. **Adjust Shooting Reward Component:** The shooting reward component has rapidly plateaued at its maximum reward value from early stages of training. This indicates that the reward for shooting might be excessively encouraging this action or maybe too easy to achieve. Consider adjusting the shooting reward mechanism to promote more strategic or effective shooting, such as rewarding successful goals more than just any shooting action.\n\n2. **Increase Variability in Dribbling and Passing Rewards:** The rewards for dribbling and passing have shown promising improvements but have quickly stabilized. This could suggest that these elements of the reward are either too easy to maximize or not sufficiently challenging. Consider adding complexity or a higher reward scaling for more complex dribbling maneuvers or successful passing in challenging situations (like under pressure or into tight spaces).\n\n3. **Base Score Reward Examination:** The base score reward mean shows zero or negative values in many cases, indicating issues either with scoring opportunities in the game or potential bugs in the reward calculation. This component of the reward needs thorough examination to ensure that it truly reflects the performance of the agents concerning their scoring abilities.\n\n4. **Enhance Reward Dynamics:** Given that the overall final reward mean shows increments and positive reinforcement learning indications, integrating more dynamics into the reward system could further enhance learning. Introducing decay elements or increasing rewards conditionally based on the match phase or opponent actions could make the training process more robust.\n\n5. **Feedback from Reward Components:** The slight fluctuations and negative dips in the final reward mean signal that there might be certain game scenarios where agents are not adequately equipped to handle or exploit efficiently. Analyzing these situations could provide insights into additional components or tweaks needed in the reward function.\n\n6. **Regular Checks for Reward Exploitation:** Ensuring ongoing monitoring of the reward system is essential to avoid potential exploitation where the agents might learn to game the system in unintended ways. Regular adjustments and checks should be part of the training regime.\n\nImplementing these changes should help in fine-tuning the reward function to better align with the training goals, ensuring a more balanced and effective learning environment for the agents."
  }

In [12]:
# This function will be used to access the Together API to generate embeddings for the movie plots

from typing import List
import numpy as np

def generate_embeddings(input_texts: List[str], model_api_string: str) -> List[List[float]]:
    """Generate embeddings from Together python library.

    Args:
        input_texts: a list of string input texts.
        model_api_string: str. An API string for a specific embedding model of your choice.

    Returns:
        embeddings_list: a list of embeddings. Each element corresponds to the each input text.
    """
    together_client = together.Together(api_key = TOGETHER_API_KEY)
    outputs = together_client.embeddings.create(
        input=input_texts,
        model=model_api_string,
    )
    return np.array([x.embedding for x in outputs.data])



In [13]:
# We will concatenate fields in the dataset in prep for embedding
  
to_embed = []

for data in rag_data:
    text = ''
    for field in ['training_goal', 'reward_function', 'suggestions']:
        value = data.get(field, '')
        text += str(value) + ' '
    to_embed.append(text.strip())
    
# Use bge-base-en-v1.5 model to generate embeddings
embeddings = generate_embeddings(to_embed, 'togethercomputer/m2-bert-80M-2k-retrieval')

可选embeddings model \n
togethercomputer/m2-bert-80M-2k-retrieval   context 2048 \n
togethercomputer/m2-bert-80M-8k-retrieval   context 8192 \n
BAAI/bge-base-en-v1.5  context 512



In [14]:
embeddings.shape, embeddings

((30, 768),
 array([[ 0.04481085,  0.07923112, -0.14739837, ..., -0.03137096,
         -0.08208837, -0.03330313],
        [ 0.08728593,  0.05275708, -0.20040941, ..., -0.06553396,
         -0.03585266, -0.06966129],
        [ 0.07489283,  0.07774817, -0.10548253, ..., -0.07664844,
         -0.00733719, -0.06739793],
        ...,
        [ 0.08953711,  0.04738197, -0.13887239, ..., -0.13070382,
         -0.03848298, -0.03208118],
        [ 0.09589171,  0.08799192, -0.15507184, ..., -0.01980245,
         -0.03123128, -0.07706562],
        [ 0.13715436,  0.02571404, -0.1366451 , ..., -0.11111125,
          0.05519155, -0.0034555 ]], shape=(30, 768)))

In [15]:
to_embed
# 目前我们已经有goal+reward funciton+改进意见的DB了，下一步是评估新的reward func并给出建议

['Focused on developing offensive strategies including mastering accurate shooting, effective dribbling to evade opponents, and practicing different pass types (long and high passes) to break defensive lines. import gym\nimport numpy as np\nclass CheckpointRewardWrapper(gym.RewardWrapper):\n    """A wrapper that adds a task-specific reward for offensive strategies focused on shooting, dribbling, and passing."""\n\n    def __init__(self, env):\n        super().__init__(env)\n        self.sticky_actions_counter = np.zeros(10, dtype=int)  # Tracks sticky actions for each agent\n\n    def reset(self):\n        """Reset the reward wrapper state along with the environment."""\n        self.sticky_actions_counter = np.zeros(10, dtype=int)\n        return self.env.reset()\n\n    def get_state(self, to_pickle):\n        """Get the state of the wrapper with additional wrapper states."""\n        to_pickle[\'sticky_actions_counter\'] = self.sticky_actions_counter\n        return self.env.get_stat

((768,),
 array([ 1.14135740e-01, -6.45996100e-01, -3.45703120e-01,  4.77539060e-01,
         2.70263670e-01, -4.64599600e-01,  9.39941400e-02,  4.45800780e-01,
        -4.06738280e-01, -6.95800800e-01,  6.46484400e-01, -4.58984380e-01,
        -4.95849600e-01,  4.34570300e-01, -2.94433600e-01,  7.75390600e-01,
        -7.47070300e-02, -1.74926760e-01,  1.85546880e-01, -5.54809570e-02,
        -1.64306640e-01,  2.55371100e-01,  5.67382800e-01,  1.76635740e-01,
        -3.98437500e-01,  1.94946290e-01, -3.56750500e-02,  1.62841800e-01,
        -7.06054700e-01,  3.23242200e-01,  7.81738300e-01, -3.24218750e-01,
        -6.63574200e-01,  3.97216800e-01, -1.08520510e-01, -6.48437500e-01,
        -9.64843750e-01, -5.27343750e-01,  2.08984380e-01,  4.42810060e-02,
        -8.78906250e-02, -3.66455080e-01,  1.35620120e-01, -1.03027340e+00,
        -5.26367200e-01,  1.31713870e-01, -2.11914060e-01, -3.14208980e-01,
        -1.05273440e+00, -2.62207030e-01,  4.79003900e-01,  1.71630860e-01,
   

In [16]:
eval_reward = """
import gym
import numpy as np
class CheckpointRewardWrapper(gym.RewardWrapper):

    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)
        self.sticky_actions_counter = np.zeros(10, dtype=int)
        # Customize these parameters as needed
        self.offensive_reward_coefficient = 0.5
        self.game_mode_change_reward = 0.3

    def reset(self):
        # Reset sticky actions counter on each reset
        self.sticky_actions_counter = np.zeros(10, dtype=int)
        return self.env.reset()

    def get_state(self, to_pickle):
        to_pickle['CheckpointRewardWrapper'] = {}
        return self.env.get_state(to_pickle)

    def set_state(self, state):
        from_pickle = self.env.set_state(state)
        # Load the state if necessary, currently we do not have a stored state
        return from_pickle

    def reward(self, reward):
        observation = self.env.unwrapped.observation()
        
        # Initialize reward components for reporting
        components = {"base_score_reward": reward.copy(),
                      "offensive_reward": [0.0] * len(reward),
                      "game_mode_change_reward": [0.0] * len(reward)}

        if observation is None:
            return reward, components

        for rew_index in range(len(reward)):
            o = observation[rew_index]
            # Give extra reward if your team possesses the ball
            if o['ball_owned_team'] == 1:
                components["offensive_reward"][rew_index] = self.offensive_reward_coefficient
                reward[rew_index] += components["offensive_reward"][rew_index]

            # Reward changes in the game mode, indicating dynamic game situations
            if o['game_mode'] != 0 and self.prev_game_mode[rew_index] == 0:
                components["game_mode_change_reward"][rew_index] = self.game_mode_change_reward
                reward[rew_index] += components["game_mode_change_reward"][rew_index]

            # Store current game mode to check for changes in the next step
            self.prev_game_mode[rew_index] = o['game_mode']

        return reward, components

    def step(self, action):
        observation, reward, done, info = self.env.step(action)
        reward, components = self.reward(reward)
        info["final_reward"] = sum(reward)
        # Add components to info for detailed logging
        for key, value in components.items():
            info[f"component_{key}"] = sum(value)
        # Tracking sticky actions
        obs = self.env.unwrapped.observation()
        self.sticky_actions_counter.fill(0)
        for agent_obs in obs:
            for i, action in enumerate(agent_obs['sticky_actions']):
                info[f"sticky_actions_{i}"] = action
        return observation, reward, done, info 
        """


In [4]:
eval_reward


'\nimport gym\nimport numpy as np\nclass CheckpointRewardWrapper(gym.RewardWrapper):\n\n    def __init__(self, env):\n        gym.RewardWrapper.__init__(self, env)\n        self.sticky_actions_counter = np.zeros(10, dtype=int)\n        # Customize these parameters as needed\n        self.offensive_reward_coefficient = 0.5\n        self.game_mode_change_reward = 0.3\n\n    def reset(self):\n        # Reset sticky actions counter on each reset\n        self.sticky_actions_counter = np.zeros(10, dtype=int)\n        return self.env.reset()\n\n    def get_state(self, to_pickle):\n        to_pickle[\'CheckpointRewardWrapper\'] = {}\n        return self.env.get_state(to_pickle)\n\n    def set_state(self, state):\n        from_pickle = self.env.set_state(state)\n        # Load the state if necessary, currently we do not have a stored state\n        return from_pickle\n\n    def reward(self, reward):\n        observation = self.env.unwrapped.observation()\n        \n        # Initialize reward 

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

# Generate the vector embeddings for the query
# 这是从LLM生成的奖励函数，或者子任务
# 随机选取rag_task_1/reward_10.py 内容用于测试
query = eval_reward

query_embedding = generate_embeddings([query], 'togethercomputer/m2-bert-80M-2k-retrieval')[0]

query_embedding.shape, query_embedding

((768,),
 array([ 2.46683530e-02,  1.04032870e-01, -1.76920300e-01,  9.75260000e-02,
        -1.64080520e-01,  8.52816000e-02, -2.82638300e-02, -2.68548130e-01,
        -2.10287760e-01,  7.25039100e-02, -2.20246130e-02,  1.51120300e-01,
        -6.97931350e-02,  4.21106960e-02, -1.49094460e-01, -1.57750830e-01,
        -1.81301060e-01,  1.43537390e-01,  9.57068950e-02,  2.33044330e-01,
         2.29456260e-02, -6.65303900e-03, -1.37341780e-01, -2.02758270e-01,
         6.04351420e-02, -1.70597880e-01,  1.05753000e-01, -1.49892630e-01,
         9.18421400e-02,  1.75622080e-01,  1.52340670e-01, -2.11572870e-01,
        -6.39805000e-02, -1.53729320e-01, -3.33504530e-01,  2.12584380e-01,
        -5.45889100e-02, -1.18705240e-01,  4.36360050e-02, -2.85970750e-02,
         1.89742430e-01, -1.70012650e-01,  1.50026750e-01,  2.84459720e-02,
        -1.35001780e-01,  1.41975090e-01,  1.42476290e-01, -2.27691010e-01,
        -1.05269864e-01, -4.39566200e-02,  4.05415860e-02,  1.66534860e-01,
   

In [18]:
# Calculate cosine similarity between the query embedding and each movie embedding
similarity_scores = cosine_similarity([query_embedding], embeddings)
similarity_scores.shape, similarity_scores

((1, 30),
 array([[0.67688898, 0.76362212, 0.65088157, 0.78161194, 0.70792639,
         0.62205228, 0.62041146, 0.72758555, 0.66038014, 0.8043457 ,
         0.68568399, 0.66914286, 0.67602504, 0.75744027, 0.73286202,
         0.69993159, 0.63754054, 0.77403406, 0.69803846, 0.67256924,
         0.69199261, 0.74515518, 0.70617896, 0.68959769, 0.66575148,
         0.5970182 , 0.71825377, 0.73122726, 0.6576758 , 0.68164149]]))

In [20]:
# 相似度检索
# Get the indices of the highest to lowest values
indices = np.argsort(-similarity_scores)

top_10_sorted_titles = [rag_data[index]['suggestions'] for index in indices[0]][:10]

top_10_sorted_titles

['1. **Adjust the Scale or Coefficient of the `base_score_reward`:**\n   - The `base_score_reward` remains mostly at zero throughout training, with occasional negative values. This lack of variation indicates that the reward function may not be rewarding the intended behaviors consistently or effectively. Consider increasing the reward for scoring goals so that the impact of this reward is more significant and provides a clearer signal to the agents.\n\n2. **Rectify and Enhance the `pass_efficiency_reward`:**\n   - The `pass_efficiency_reward` shows some increase during training but overall contributes minimally to the final reward. To make this component more influential in teaching offensive strategies, consider increasing the reward scale for successful passes, particularly for high and long passes that are specifically targeted in your training goals. This would likely make the reward signal stronger and more aligned with the training objective.\n\n3. **Expand the Diversity of Beha

In [22]:
# 封装retrieval
def retrieve(query: str, top_k: int = 5, index: np.ndarray = None) -> List[int]:
    """
    Retrieve the top-k most similar items from an index based on a query.
    Args:
        query (str): The query string to search for.
        top_k (int, optional): The number of top similar items to retrieve. Defaults to 5.
        index (np.ndarray, optional): The index array containing embeddings to search against. Defaults to None.
    Returns:
        List[int]: A list of indices corresponding to the top-k most similar items in the index.
    """
    
    query_embedding = generate_embeddings([query], 'togethercomputer/m2-bert-80M-2k-retrieval')[0]
    similarity_scores = cosine_similarity([query_embedding], index)

    return np.argsort(-similarity_scores)[0][:top_k]

In [23]:
retrieve(eval_reward, top_k=5, index = embeddings)

array([ 9,  3, 17,  1, 13])

In [ ]:
# Generation过程
# Extract out the titles and overviews of the top 10 most similar movies
# titles = [rag_data[index]['title'] for index in indices[0]][:10]
# overviews = [rag_data[index]['overview'] for index in indices[0]][:10]

client = Together(api_key = TOGETHER_API_KEY)

# Generate a story based on the top 10 most similar movies

# 试一下非reasoning和reasoning LLM

response = client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[
      {
        "role": "system", 
        "content": "You are a great reward designers for RL tasks google research football. You must respond in a specific format. Please answer in the following format: \n**Evaluation:**\n**Suggestions:**\n**In which, 'Evaluation' represents the assessment result and should only be either 'Yes' or 'No,' indicating whether this reward function can help the agent achieve its training goals."},
      {
        "role": "user", 
        "content": f"This is the current designed reward functions {query}, please give suggestions"},
    ],
)

print(response.choices[0].message.content)

**Evaluation:** No

**Suggestions:**

1. Consider adding a cap to the maximum reward that can be given in a single step to prevent the agent from receiving an extremely high reward and to encourage more balanced behavior.
2. The reward function seems to be focused on the team that possesses the ball, which might not be the best approach. Consider adding rewards for other aspects of the game, such as scoring goals or preventing the opponent from scoring.
3. The game mode change reward might not be the best way to encourage the agent to adapt to changing game situations. Consider adding more specific rewards for different game modes or situations.
4. The sticky actions counter might not be the best way to track the agent's behavior. Consider using a more sophisticated method, such as tracking the frequency or duration of certain actions.
5. The reward function could be improved by incorporating more domain knowledge about the game of football. For example, rewards could be given for cert

### Discussion
1. Suggestions 
2. Evaluation reward traj
3. State-action Traj
